In [1]:
import sys
sys.path.append('../../core/')

from miniML import MiniTrace, EventDetection
from miniML_plot_functions import miniML_plots
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

In [2]:
model_1 = '../../models/GC_lstm_model.h5'
model_2 = '../../models/transfer_learning/GC_mEPSP/lstm_transfer.h5'

In [3]:
filename = '../../example_data/B_2020-05-14_007.dat'

win_size = 3000
threshold = 0.5
event_direction = 'positive'
training_direction = 'negative'

rectype = 'conti CC'
scaling = 1e3
unit = 'mV'

trace = MiniTrace.from_heka_file(filename=filename,
                                 rectype=rectype,
                                 scaling=scaling,
                                 unit=unit)

In [4]:
detection_1 = EventDetection(data=trace,
                           model_path=model_1,
                           window_size=win_size,
                           model_threshold=threshold,
                           batch_size=512,
                           event_direction=event_direction,
                           training_direction=training_direction,
                           compile_model=True)

detection_1.detect_events(eval=True, convolve_win=20, rel_prom_cutoff=0.75)

Model loaded from ../../models/GC_lstm_model.h5
118/118 [==============================] - 4s 27ms/step

Event statistics:
-------------------------
    Number of events: 79
    Average score: 0.847
    Event frequency: 0.6583 Hz
    Mean amplitude: 1.0204 mV
    Median amplitude: 0.4077 mV
    Std amplitude: 1.3295 mV
    CV amplitude: 1.303
    Mean charge: 0.00542 pC
    CV charge: 1.090
    Mean 10-90 risetime: 1.793 ms
    Mean half decay time: 2.384 ms
    Tau decay: 3.886 ms
-------------------------


In [5]:
detection_2 = EventDetection(data=trace,
                           model_path=model_2,
                           window_size=win_size,
                           model_threshold=threshold,
                           batch_size=512,
                           event_direction=event_direction,
                           training_direction=training_direction,
                           compile_model=True)

detection_2.detect_events(eval=True, convolve_win=20, rel_prom_cutoff=0.75)

Model loaded from ../../models/transfer_learning/GC_mEPSP/lstm_transfer.h5
118/118 [==============================] - 4s 27ms/step

Event statistics:
-------------------------
    Number of events: 23
    Average score: 0.969
    Event frequency: 0.1917 Hz
    Mean amplitude: 2.6514 mV
    Median amplitude: 2.6035 mV
    Std amplitude: 1.5165 mV
    CV amplitude: 0.572
    Mean charge: 0.01277 pC
    CV charge: 0.476
    Mean 10-90 risetime: 1.029 ms
    Mean half decay time: 2.777 ms
    Tau decay: 3.608 ms
-------------------------


In [6]:
%matplotlib qt
fig, axs = plt.subplots(3, 1, sharex=True)

prediction_x = np.arange(0, len(detection_1.prediction)) * trace.sampling


axs[0].plot(prediction_x, detection_1.prediction, c='k', alpha=0.7, label=f'{model_1}')
axs[0].plot(prediction_x, detection_2.prediction, c='b', alpha=0.7, label=f'{model_2}')
axs[0].legend(loc='upper right')

axs[1].plot(trace.time_axis, trace.data, c='k')
axs[1].scatter(detection_1.event_peak_times, trace.data[detection_1.event_peak_locations], c='orange', zorder=2)

axs[2].plot(trace.time_axis, trace.data, c='b')
axs[2].scatter(detection_2.event_peak_times, trace.data[detection_2.event_peak_locations], c='orange', zorder=2)

axs[1].sharey(axs[2])

plt.show()